In [85]:
from pathlib import Path

import pandas as pd
from promg.modules.db_management import DBManagement
from promg import Configuration, DatabaseConnection, Performance, SemanticHeader, DatasetDescriptions, OcedPg, Query

import yaml

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

In [2]:
case_study = 'bpic14'

conf_path = Path(case_study, 'config.yaml')
config = yaml.safe_load(open(conf_path))

print(f"These are the credentials that I expect to be set for the database.")
print(f"db_name: {config['db_name']}")
print(f"uri: {config['uri']}")
print(f"password: {config['password']}")
print("----------------------")
print(f"If you have other credentials, please change them at: {conf_path}")

config = Configuration.init_conf_with_config_file(conf_path)
db_connection = DatabaseConnection.set_up_connection(config=config)

These are the credentials that I expect to be set for the database.
db_name: neo4j
uri: bolt://localhost:7687
password: bpic2014
----------------------
If you have other credentials, please change them at: bpic14\config.yaml


In [ ]:
# clear previously created nodes and relations
# start/end relations
qClearStart = f'''MATCH (n)<-[r:START]-(e:Event) DELETE r'''
db_connection.exec_query(qClearStart)
qClearEnd = f'''MATCH (n)<-[r:END]-(e:Event) DELETE r'''
db_connection.exec_query(qClearEnd)
# high-level events
qClearStartHLE = f'''MATCH (n)<-[r:START]-(e:HighLevelEvent) DELETE r'''
db_connection.exec_query(qClearStartHLE)
qClearEndHLE = f'''MATCH (n)<-[r:END]-(e:HighLevelEvent) DELETE r'''
db_connection.exec_query(qClearEndHLE)
qClearHighLevelEvent = f'''MATCH (h:HighLevelEvent) DETACH DELETE h'''
db_connection.exec_query(qClearHighLevelEvent)
qClearHL_DF = f'''MATCH (:Event)-[r:HL_DF]-(e:Event) DELETE r'''
db_connection.exec_query(qClearHL_DF)

In [11]:
qClearStart = '''
:auto
MATCH (n)<-[r:START_CORR]-(e:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''

db_connection.exec_query(qClearStart)
qClearEnd = '''
:auto
MATCH (n)<-[r:END_CORR]-(e:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''
db_connection.exec_query(qClearEnd)

[]

In [31]:
qClearDuring = '''
:auto
MATCH (:HighLevelEvent)<-[r:DURING]-(:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''
db_connection.exec_query(qClearDuring)

[]

In [10]:
# fix DF relations for incidents
qDeleteDFIncidents = f'''
    MATCH (e1:Event)-[df:DF {{objectType:"Incident"}}]->(e2:Event)
    DELETE df
'''
db_connection.exec_query(qDeleteDFIncidents)

with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qBuildDFIncidents = f'''
        MATCH (e:Event)-[:CORR]->(o:Incident)
        WHERE e.timestamp IS NOT NULL
        WITH o, e ORDER BY e.timestamp, ID(e)
        WITH o, collect(e) AS events
        UNWIND range(0, size(events)-2) AS idx
        WITH events[idx] AS fromEv, events[idx+1] AS toEv, o.sysId AS objSysId
        MERGE (fromEv)-[rel:DF {{objectType:"Incident", id:objSysId}}]->(toEv)
    '''
    result = session.run(qBuildDFIncidents)
    result.consume()
    print(str(result.single))

<bound method Result.single of <neo4j._sync.work.result.Result object at 0x00000221C9219910>>


In [5]:
objectTypes = ["Change", "Incident", "Interaction"]

In [4]:
# infer start and end events for each object type

for objectType in objectTypes:
    print(f"Inferring start and end events for object type: {objectType}")

    qStartEvent = f'''
    // Infer start event of an object
    MATCH (n:{objectType})
    MATCH (n)<-[c:CORR]-(e:Event) WHERE NOT ()-[:DF {{id:n.sysId}}]->(e)
    MERGE (n)<-[:START]-(e)
    '''
    result = db_connection.exec_query(qStartEvent)
    qEndEvent = f'''
    // Infer start event of an object
    MATCH (n:{objectType})
    MATCH (n)<-[c:CORR]-(e:Event) WHERE NOT (e)-[:DF {{id:n.sysId}}]->()
    MERGE (n)<-[:END]-(e)
    '''
    db_connection.exec_query(qEndEvent)

Inferring start and end events for object type: Change
Inferring start and end events for object type: Incident
Inferring start and end events for object type: Interaction


In [5]:
# materialize time between events as duration on DF edge
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    q_materialize_duration_on_df = f'''
        MATCH (e1:Event)-[df:DF]->(e2:Event) WHERE df.duration IS NOT NULL
        CALL {{
            WITH e1,e2,df
            SET df.duration = duration.inSeconds(e1.timestamp, e2.timestamp).seconds    
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(q_materialize_duration_on_df)
    result.consume()
    print(str(result.single))

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (e1, e2, df) { ... }', position=<SummaryInputPosition line=3, column=9, offset=84>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 84, 'line': 3, 'column': 9}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (e1:Event)-[df:DF]->(e2:Event) WHERE df.duration IS NOT NULL\n        CALL {\n            WITH e1,e2,df\n            SET df.duration = duration.inSeconds(e1.timestamp, e2.timestamp).seconds    \n        } IN TRANSACTIONS OF 1000 ROWS\n        RETURN count(*) AS total\n    '


<bound method Result.single of <neo4j._sync.work.result.Result object at 0x000001A638B528D0>>


In [7]:
# build index for high-level events
qCreateIndexHLE_sys = f'''
    CREATE INDEX highLevelEventSysIdIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.sysId);
'''
db_connection.exec_query(qCreateIndexHLE_sys)
qCreateIndexHLE_start = f'''
    CREATE INDEX highLevelEventStartTimeIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.startTime);
'''
db_connection.exec_query(qCreateIndexHLE_start)
qCreateIndexHLE_end = f'''
    CREATE INDEX highLevelEventEndTimeIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.endTime);
'''
db_connection.exec_query(qCreateIndexHLE_end)

[]

In [13]:
# build high-level events
qBuildHighLevelEvent = '''
    :auto
    MATCH (n) - [:IS_OF_TYPE] -> (ot:ObjectType {objectType: $objectType})
    MATCH (eStart:Event)-[st:START]->(n)<-[en:END]-(eEnd:Event)
    WITH DISTINCT eStart, eEnd
    CALL (eStart, eEnd) {
        MERGE (h:HighLevelEvent {sysId: eStart.sysId + "_" + eEnd.sysId})
        ON CREATE SET h.startTime=eStart.timestamp, h.endTime=eEnd.timestamp, h.activity=$objectType
        MERGE (h)-[:START]->(eStart)
        MERGE (h)-[:END]->(eEnd)
    } IN TRANSACTIONS
'''

for objectType in objectTypes:
    query = Query(
        query_str=qBuildHighLevelEvent,
        parameters={
            "objectType": objectType,
        }
    )

    db_connection.exec_query(query)

In [15]:
# lift CORR to high-level events
qLiftCorrToHL = '''
    :auto
    MATCH (h:HighLevelEvent)-[:START]->(eStart:Event)-[:CORR]->(n)
    CALL (h, n) {
        MERGE(h) - [:CORR]->(n)
    } IN TRANSACTIONS
'''
db_connection.exec_query(qLiftCorrToHL)

[]

In [3]:
# build DF relations between high-level events
# step 1: build HL-DF relations between events

qBuildDfRelationsHLE = '''
    :auto
    // get only START/END events of each HL event correlated to CI_SC
    MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (o) <-[:CORR]-(h:HighLevelEvent)-->(e:Event)
    // order by time
    WHERE e.timestamp IS NOT NULL
    WITH o, e ORDER BY e.timestamp, ID(e)
    WITH o, collect(e) AS events
    UNWIND range(0, size(events)-2) AS idx
    WITH events[idx] AS fromEv, events[idx+1] AS toEv, o
    CALL (fromEv,toEv,o) {
        // get their HL events correlated to the same object
        MATCH (fromEv)<--(h1:HighLevelEvent)-[:CORR]->(o)<-[:CORR]-(h2:HighLevelEvent)-->(toEv) WHERE h1 <> h2
        // and add their DF relation
        MERGE (h1)-[rel:DF {objectType:$objectType, id:o.sysId}]->(h2)
    } IN TRANSACTIONS
    RETURN count(*) AS total
'''

In [6]:
for objectType in objectTypes:
    query = Query(
        query_str=qBuildDfRelationsHLE,
        parameters={
            "objectType": objectType,
        }
    )

    db_connection.exec_query(query)

In [18]:
# infer start and end events for each object type

object_type = "CI_SC"
print(f"Inferring start and end events for object type: {object_type}")

q_start_event_str = '''
:auto
// Infer start event of an object
MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (n)
CALL (n) {
    MATCH (n) <- [:CORR] - (e:HighLevelEvent)
    RETURN e ORDER BY e.startTime, id(e) LIMIT 1
}
CALL (n, e) {
    MERGE (n)<-[:START_CORR]-(e)
} IN TRANSACTIONS
RETURN count(n)
'''

q_start_event = Query(
    query_str=q_start_event_str,
    parameters={
        "objectType": object_type,
    }
)

db_connection.exec_query(q_start_event)

Inferring start and end events for object type: CI_SC


[{'count(n)': 22259}]

In [24]:


q_end_event_str = '''
:auto
// Infer end event of an object
MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (n)
CALL (n) {
    MATCH (n) <- [:CORR] - (e:HighLevelEvent)
    RETURN e ORDER BY e.endTime DESC, id(e) LIMIT 1
}
CALL (n, e) {
    MERGE (n)<-[:END_CORR]-(e)
} IN TRANSACTIONS

RETURN count(n)
'''

q_end_event = Query(
    query_str=q_end_event_str,
    parameters={
        "objectType": object_type,
    }
)

db_connection.exec_query(q_end_event)

[{'count(n)': 22259}]

In [131]:
# get the bag variants on the high_level
q_number_of_events = '''
MATCH (ci_sc:CI_SC) <- [:CORR] - (e:HighLevelEvent)
MATCH (ci_sc) - [:RELATED_CI] -> (ci:ConfigurationItem)
WHERE ci.ciType is not null AND ci.ciSubtype is not null AND ci.ciType <> "#N/B"
WITH ci_sc, ci, e.activity AS activity, count(e) as event_count ORDER BY activity
RETURN ci_sc.sysId as sysId, ci.ciType as type, ci.ciSubtype as subtype, activity, event_count
'''

result = pd.DataFrame(db_connection.exec_query(q_number_of_events))
result

,sysId,type,subtype,activity,event_count
0,LSR000699_WBS000161,computer,Linux Server,Change,2
1,DTA000266_WBS000135,application,Desktop Application,Change,2
2,HMD000002_WBS000195,hardware,MigratieDummy,Change,6
3,WSR000561_WBS000102,computer,Windows Server,Change,1
4,SUB000494_WBS000162,subapplication,Web Based Application,Change,4
...,...,...,...,...,...
15986,SBA000571_WBS000231,application,Server Based Application,Interaction,20
15987,SBA000060_WBS000199,application,Server Based Application,Interaction,38
15988,WSR001619_WBS000102,computer,Windows Server,Interaction,1
15989,WSR000834_WBS000102,computer,Windows Server,Interaction,1


In [118]:
# Pivot the DataFrame
pivot_df = result.pivot(
    index=['sysId', 'type', 'subtype'],
    columns='activity',
    values='event_count'
).reset_index()

pivot_df = pivot_df.fillna(0)
pivot_df

activity,sysId,type,subtype,Change,Incident,Interaction
0,ACS000001_WBS000252,applicationcomponent,Application Server,0.0,1.0,1.0
1,ADB000001_WBS000253,database,Applicatie Database,2.0,0.0,0.0
2,ADB000002_WBS000253,database,Applicatie Database,1.0,0.0,0.0
3,ADB000003_WBS000253,database,Applicatie Database,2.0,0.0,0.0
4,ADB000004_WBS000253,database,Applicatie Database,5.0,0.0,0.0
...,...,...,...,...,...,...
12788,ZOS000029_WBS000199,computer,zOS Server,1.0,0.0,0.0
12789,ZOS000030_WBS000199,computer,zOS Server,1.0,0.0,0.0
12790,ZOS000031_WBS000199,computer,zOS Server,1.0,0.0,0.0
12791,ZOS000032_WBS000199,computer,zOS Server,1.0,0.0,0.0


In [159]:
compare_df = pivot_df
def get_activity_order(row):
    activities = ["Change", "Interaction", "Incident"]
    sorted_activities = sorted(zip(activities, row[activities]), key=lambda x: x[1], reverse=True)
    return " >= ".join([f"{activity} ({value})" for activity, value in sorted_activities if value > 0])

# Apply the function to each row
compare_df["order"] = compare_df.apply(get_activity_order, axis=1)

# Display the updated DataFrame
# Group by 'app_wbs' and 'type', then aggregate by 'subtype' to find min and max
result = compare_df.groupby(['type', 'order']).agg(
    {'sysId': 'count'})
result['%'] = (round(result.sysId / result.groupby(level=0).sysId.transform("sum") * 100, 2))
result

activity                                                           sysId  \
type           order                                                       
Phone          Change (1.0)                                          117   
               Change (2.0)                                            3   
               Incident (1.0)                                          1   
               Interaction (1.0)                                       1   
               Interaction (1.0) >= Incident (1.0)                     1   
...                                                                  ...   
subapplication Interaction (776.0) >= Incident (259.0)                 1   
               Interaction (839.0) >= Incident (160.0)                 1   
               Interaction (89.0) >= Incident (52.0) >= Change...      1   
               Interaction (926.0) >= Incident (228.0) >= Chan...      1   
               Interaction (99.0) >= Incident (85.0) >= Change...      1   

activity                                                               %  
type           order                                                      
Phone          Change (1.0)                                        95.12  
               Change (2.0)                                         2.44  
               Incident (1.0)                                       0.81  
               Interaction (1.0)                                    0.81  
               Interaction (1.0) >= Incident (1.0)                  0.81  
...                                                                  ...  
subapplication Interaction (776.0) >= Incident (259.0)              0.20  
               Interaction (839.0) >= Incident (160.0)              0.20  
               Interaction (89.0) >= Incident (52.0) >= Change...   0.20  
               Interaction (926.0) >= Incident (228.0) >= Chan...   0.20  
               Interaction (99.0) >= Incident (85.0) >= Change...   0.20  

[1045 rows x 2 columns]

In [142]:
import seaborn as sns

fig = px.box(data_frame = result, x="type", y="event_count",  color="activity")
fig.show()

In [144]:
# Group by 'app_wbs' and 'type', then aggregate by 'subtype' to find min and max
result = pivot_df.groupby(['type']).agg(
    {'sysId': 'count',
     'Change': 'max',
     'Incident': 'max',
     'Interaction': 'max'})
result

activity,sysId,Change,Incident,Interaction
type,,,,
Phone,123,2.0,1.0,1.0
application,1702,101.0,2172.0,6884.0
applicationcomponent,408,33.0,3.0,3.0
computer,7052,52.0,150.0,783.0
database,574,296.0,171.0,225.0
displaydevice,27,1.0,104.0,1477.0
hardware,235,38.0,108.0,112.0
networkcomponents,1004,730.0,2.0,10.0
no type,19,2.0,50.0,478.0


In [37]:
table_reset = table.reset_index()
table_reset[table_reset['type'] == 'applicationcomponent']

,type,subtype,bag_variant,num_ci_sc,%
48,applicationcomponent,Application Server,Incident-Interaction,1,100.0%
49,applicationcomponent,MQ Queue Manager,Change,405,99.51%
50,applicationcomponent,MQ Queue Manager,Change-Incident-Interaction,1,0.25%
51,applicationcomponent,MQ Queue Manager,Incident-Interaction,1,0.25%


In [45]:
# get the bag variants on the high_level
q_set_variants = '''
MATCH (ci_sc:CI_SC) <- [:CORR] - (e:HighLevelEvent)
MATCH (ci_sc) - [:RELATED_CI] -> (ci:ConfigurationItem)
WITH ci_sc, ci, e.activity AS activity, count(e) as cnt_act ORDER BY activity
WITH ci_sc.sysId as sysId, ci.ciType as type, ci.ciSubtype as subtype, collect(distinct activity + " (" + cnt_act + ")") as bag_variant
RETURN type, subtype, rtrim(reduce(str = "", act in bag_variant | str + act + " - ")," - ") as set_variant, count(distinct sysId) as num_ci_sc
'''

result_set = pd.DataFrame(db_connection.exec_query(q_set_variants))
result_set

,type,subtype,set_variant,num_ci_sc
0,computer,Linux Server,Change (2),268
1,application,Desktop Application,Change (2),126
2,hardware,MigratieDummy,Change (6) - Incident (2) - Interaction (2),1
3,computer,Windows Server,Change (1),879
4,subapplication,Web Based Application,Change (4),2
...,...,...,...,...
1505,computer,Desktop,Interaction (4),1
1506,subapplication,Server Based Application,Interaction (3),1
1507,application,Citrix,Interaction (1),1
1508,networkcomponents,Net Device,Interaction (1),1


In [43]:
table = pd.pivot_table(result_set, index=['type', 'subtype', 'set_variant'], aggfunc="sum")
table['%'] = (round(table.num_ci_sc / table.groupby(level=1).num_ci_sc.transform("sum") * 100, 2)).astype(str) + '%'
table

num_ci_sc  \
type           subtype               set_variant                                    
#N/B           #N/B                  Interaction (16)                           1   
Phone          Number                Change (1)                               117   
                                     Change (2)                                 3   
                                     Incident (1)                               1   
                                     Incident (1) -Interaction (1)              1   
...                                                                           ...   
subapplication Web Based Application Incident (8) -Interaction (19)             1   
                                     Incident (90) -Interaction (208)           1   
                                     Incident (96) -Interaction (195)           1   
                                     Interaction (1)                            7   
                                     Interaction (5)                            1   

                                                                             %  
type           subtype               set_variant                                
#N/B           #N/B                  Interaction (16)                   100.0%  
Phone          Number                Change (1)                         95.12%  
                                     Change (2)                          2.44%  
                                     Incident (1)                        0.81%  
                                     Incident (1) -Interaction (1)       0.81%  
...                                                                        ...  
subapplication Web Based Application Incident (8) -Interaction (19)       0.4%  
                                     Incident (90) -Interaction (208)     0.4%  
                                     Incident (96) -Interaction (195)     0.4%  
                                     Interaction (1)                      2.8%  
                                     Interaction (5)                      0.4%  

[1498 rows x 2 columns]

In [46]:
table_reset = table.reset_index()
table_reset[table_reset['type'] == 'applicationcomponent']

,type,subtype,set_variant,num_ci_sc,%
551,applicationcomponent,Application Server,Incident (1) -Interaction (1),1,100.0%
552,applicationcomponent,MQ Queue Manager,Change (1),186,45.7%
553,applicationcomponent,MQ Queue Manager,Change (11),1,0.25%
554,applicationcomponent,MQ Queue Manager,Change (13),1,0.25%
555,applicationcomponent,MQ Queue Manager,Change (14),1,0.25%
556,applicationcomponent,MQ Queue Manager,Change (17),1,0.25%
557,applicationcomponent,MQ Queue Manager,Change (18),1,0.25%
558,applicationcomponent,MQ Queue Manager,Change (19),1,0.25%
559,applicationcomponent,MQ Queue Manager,Change (2),105,25.8%
560,applicationcomponent,MQ Queue Manager,Change (21),1,0.25%


In [47]:
# get the bag variants on the high_level
q_set_variants = '''
MATCH (ci_sc:CI_SC) <- [:CORR] - (e:HighLevelEvent)
MATCH (ci_sc) - [:RELATED_CI] -> (ci:ConfigurationItem {ciType: 'applicationcomponent'})
MATCH (e) - [:CORR] -> (c:Change)
WITH ci_sc, ci, c order by c.sysId
WITH ci_sc.sysId as sysId, ci.ciType as type, ci.ciSubtype as subtype, collect(distinct c.sysId) as change_variant
RETURN type, subtype, rtrim(reduce(str = "", act in change_variant | str + act + " - ")," - ") as change_variant, count(distinct sysId) as num_ci_sc
'''

result_set = pd.DataFrame(db_connection.exec_query(q_set_variants))
result_set

,type,subtype,change_variant,num_ci_sc
0,applicationcomponent,MQ Queue Manager,C00000247 - C00003105 - C00004690 - C00005518 ...,1
1,applicationcomponent,MQ Queue Manager,C00001071 - C00005916,1
2,applicationcomponent,MQ Queue Manager,C00001071 - C00013961 - C00014992 - C00016357,2
3,applicationcomponent,MQ Queue Manager,C00001071 - C00003995 - C00010947 - C00011714 ...,1
4,applicationcomponent,MQ Queue Manager,C00001217 - C00007050,2
...,...,...,...,...
228,applicationcomponent,MQ Queue Manager,C00017325,10
229,applicationcomponent,MQ Queue Manager,C00017327,2
230,applicationcomponent,MQ Queue Manager,C00017633 - C00017871,1
231,applicationcomponent,MQ Queue Manager,C00017700 - C00017871,3


In [3]:
# find overlapping high-level events and create DURING relations

qFindContainedHLEvents = '''
:auto
    MATCH (h1:HighLevelEvent) - [df:DF] -> (h2:HighLevelEvent) - [:DF {id: df.id}] -> (h1)
    WHERE h1 <> h2 AND h1.startTime < h2.startTime AND h2.endTime < h1.endTime
      CALL (h1, h2) {
        MERGE (h2)-[:DURING]->(h1)
    } IN TRANSACTIONS
    RETURN count(h2) AS total
'''
db_connection.exec_query(qFindContainedHLEvents)

[{'total': 18823}]

In [ ]:
# find overlapping high-level events and create DURING relations

qFindContainedHLEvents = '''
:auto
    MATCH (h1:HighLevelEvent) - [df:DF] -> (h2:HighLevelEvent) - [df2:DF {id: df.id}] -> (h1)
    WHERE h1 <> h2 AND h1.startTime < h2.startTime AND h2.endTime < h1.endTime
      CALL (h1, h2) {
        MERGE (h2)-[:DURING]->(h1)
    } IN TRANSACTIONS
    RETURN count(h2) AS total
'''
db_connection.exec_query(qFindContainedHLEvents)

In [ ]:
# lift DF relations from events to high-level events
qLiftDfRelations = '''
    :auto
    MATCH (h1:HighLevelEvent)-[:END]->(e1:Event)-[df:DF]->(e2:Event)<-[:START]-(h2:HighLevelEvent)
    CALL (h1, df, h2) {
        MERGE (h1)-[df2:DF]->(h2)
        ON CREATE SET df2 = properties(df)
    }
    YIELD total
    RETURN total
'''
result = db_connection.exec_query(qLiftDfRelations)
print(result)



In [ ]:
# lift DF relations from events to high-level events
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qLiftDfRelations = f'''
        MATCH (h1:HighLevelEvent)-[:END]->(e1:Event)-[df:DF]->(e2:Event)<-[:START]-(h2:HighLevelEvent)
        CALL {{
            WITH h1,df,h2
            MERGE (h1)-[df2:DF]->(h2) ON CREATE SET df2 = properties(df)
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(qLiftDfRelations)
    result.consume()
    print(result.single)

In [ ]:
# find overlapping high-level events and create DURING relations
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qFindContainedHLEvents = f'''
        MATCH (h1:HighLevelEvent)-[c1:CORR]->(n:CI_SC)<-[c2:CORR]-(h2:HighLevelEvent) WHERE h1 <> h2
        WHERE h1.startTime < h2.startTime AND h2.endTime < h1.endTime
        CALL {{
            WITH h1,h2
            MERGE (h2)-[:DURING]->(h1)
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(qFindContainedHLEvents)
    result.consume()
    print(result.single)